In [1]:
import cv2
import numpy as np
import os
import time
import warnings
from sklearn.metrics.pairwise import cosine_similarity
import sys
import re
from nltk import download
import pandas as pd
from tqdm import tqdm
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import pos_tag, RegexpParser, word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install pyLDAvis
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk

# Download NLTK resources
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

from PyPDF2 import PdfReader
from PIL import ImageGrab, Image
import fitz
import pytesseract
from pdf2image import convert_from_path
import pyautogui

# This Python environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here are several helpful packages to load in
import easyocr
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g., pd.read_csv)
import re
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
C:\Users\hamma\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\hamma\AppData\Local\Tesseract-OCR\tesseract.exe'

In [3]:
import re
import pandas as pd

def extract_concepts_and_definitions_from_text(text_file):
    with open(text_file, 'r', encoding='utf-8') as file:
        text = file.read()

    concepts_and_definitions = []
    current_concept = ""
    current_concept_name = ""
    current_definition = ""

    lines = text.split('\n')

    for line in lines:
        # Check if the line matches the pattern for a concept
        if re.match(r'(\d+(\.\d+)*)\s+(.+)', line):
            if current_concept:
                # Append the current concept and definition to the list as a tuple
                concepts_and_definitions.append((current_concept, current_concept_name, current_definition))

            # Split the line into concept and definition
            concept_match = re.match(r'(\d+(\.\d+)*)\s+(.+)', line)
            current_concept = concept_match.group(1)
            current_concept_name = concept_match.group(3)
            current_definition = ""
        else:
            # If the line does not match the pattern, consider it part of the definition
            current_definition += "\n" + line

    # Append the last concept and definition to the list
    concepts_and_definitions.append((current_concept, current_concept_name, current_definition))

    # Create a DataFrame
    df = pd.DataFrame(concepts_and_definitions, columns=["Number", "Concept", "Description"])
    return df

# Example of usage
text_file_path = 'ocr_results_PMBOOKimgs.txt'  # Replace with the path to your text file
df = extract_concepts_and_definitions_from_text(text_file_path)

# Print or process the DataFrame
print(df)


       Number                                Concept  \
0        11.1                   Plan Risk Management   
1        11.2                         Identify Risks   
2        11.3      Perform Qualitative Risk Analysis   
3        11.4     Perform Quantitative Risk Analysis   
4        11.5                    Plan Risk Responses   
..        ...                                    ...   
113  11.7.3.1           WORK PERFORMANCE INFORMATION   
114  11.7.3.2                        CHANGE REQUESTS   
115  11.7.3.3        PROJECT MANAGEMENT PLAN UPDATES   
116  11.7.3.4              PROJECT DOCUMENTS UPDATES   
117  11.7.3.5  ORGANIZATIONAL PROCESS ASSETS UPDATES   

                                           Description  
0    \nThe process of defining how to conduct risk ...  
1    \nThe process of identifying individual projec...  
2    \nThe process of prioritizing individual proje...  
3    \nThe process ofnumerically analyzing the comb...  
4    \nThe process of developing options, 

In [4]:
df

,Number,Concept,Description
0,11.1,Plan Risk Management,\nThe process of defining how to conduct risk ...
1,11.2,Identify Risks,\nThe process of identifying individual projec...
2,11.3,Perform Qualitative Risk Analysis,\nThe process of prioritizing individual proje...
3,11.4,Perform Quantitative Risk Analysis,\nThe process ofnumerically analyzing the comb...
4,11.5,Plan Risk Responses,"\nThe process of developing options, selecting..."
...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,\nDescribed in Section 4.5.1.3. Work performan...
114,11.7.3.2,CHANGE REQUESTS,\nY\nDescribed in Section 4.3.3.4. The Monitor...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nProject documents that may be updated as a r...


In [5]:
# Apply text cleaning to the "Definition" column
df["Description"] = df["Description"].apply(lambda x: x.lower())

In [6]:
df

,Number,Concept,Description
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...
1,11.2,Identify Risks,\nthe process of identifying individual projec...
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting..."
...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,\ndescribed in section 4.5.1.3. work performan...
114,11.7.3.2,CHANGE REQUESTS,\ny\ndescribed in section 4.3.3.4. the monitor...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...


In [7]:
# Function to extract "Described In" values
def extract_described_in(definition):
    described_in_matches = re.findall(r'(?:\(?)described in section (\d+(\.\d+)*)', definition)
    if described_in_matches:
        return ', '.join([match[0] for match in described_in_matches])
    else:
        return None
# Apply the function to the DataFrame
df['Described In'] = df['Description'].apply(extract_described_in)

# Display the DataFrame
df

,Number,Concept,Description,Described In
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None
...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,\ndescribed in section 4.5.1.3. work performan...,4.5.1.3
114,11.7.3.2,CHANGE REQUESTS,\ny\ndescribed in section 4.3.3.4. the monitor...,4.3.3.4
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2"


In [8]:
# Function to remove "described in" part
def remove_described_in(definition):
    return re.sub(r'\(?\s*described in section \d+(\.\d+)*\s*\)?', '', definition)
# Apply the function to the "Definition" column
df['Description'] = df['Description'].apply(remove_described_in)
# Display the DataFrame
df

,Number,Concept,Description,Described In
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None
...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2"


In [9]:
def extract_figures(definition):
    figure_matches = re.findall(r'figure (\d{1,2}-\d{1,2}[a-zA-Z]{0,6})', definition)
    if figure_matches:
        figures = ', '.join(figure_matches)
        return figures
    else:
        return None
# Apply the function to extract figures and add them to the "Figure" column
df['Figure'] = df['Description'].apply(extract_figures)

# Display the DataFrame
df

,Number,Concept,Description,Described In,Figure
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None
...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None


In [10]:
import pandas as pd

# Filter the original DataFrame to get rows with non-None values in the "Figure" column
non_none_figures_df = df[df['Figure'].notna()]

# Display the resulting DataFrame
non_none_figures_df

,Number,Concept,Description,Described In,Figure
6,11.7,Monitor Risks,\nthe process of monitoring the implementation...,None,11-1
7,11.1,PLAN RISK MANAGEMENT,\nplan risk management is the process of defin...,None,"11-3, 11-2, 11-3"
19,11.1.3.1,RISK MANAGEMENT PLAN,\nthe risk management plan is a component of t...,11.3.2.6,"11-4, 11-5, 11-5"
20,11.2,IDENTIFY RISKS,\nidentify risks is the process of identifying...,None,"11-6, 11-7, 11-7"
22,11.2.1.1,PROJECT MANAGEMENT PLAN,. project management plan components include b...,"4.2.3.1, 6.1.3.1, 7.1.3.1, 8.1.3.1, 9.1.3.1, 1...",11-4
39,11.3,PERFORM QUALITATIVE RISK ANALYSIS,\nperform qualitative risk analysis is the pro...,None,"11-8, 11-9, 11-8, 11-9"
50,11.3.2.5,RISK CATEGORIZATION,\nrisks to the project can be categorized by s...,None,11-4
51,11.3.2.6,DATA REPRESENTATION,\ndata representation techniques that can be u...,None,"11-5, 11-10, 11-10"
55,11.4,PERFORM QUANTITATIVE RISK ANALYSIS,\nperform quantitative risk analysis is the pr...,None,"11-11, 11-12, 11-11, 11-12"
66,11.4.2.5,DATA ANALYSIS,\ndata analysis techniques that can be used du...,None,"11-13, 11-13, 11-14, 11-14, 11-15, 11-15"


In [11]:
import os

# Specify the directory path where your figures are located
figures_directory = "PMBOOKimgs/"

# Function to generate figure URLs
def generate_figure_url(figure_number):
    # Construct the filename based on the figure number
    filename = f"figure{figure_number}.png"
    # Construct the full file URL using the directory path
    figure_url = os.path.join(figures_directory, filename)
    return figure_url

# Apply the function to the "Figure" column to create the "Figure_URL" column
df['Figure'] = df['Figure'].apply(generate_figure_url)

# Display the DataFrame with figure URLs
df

,Number,Concept,Description,Described In,Figure
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,PMBOOKimgs/figureNone.png
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,PMBOOKimgs/figureNone.png
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,PMBOOKimgs/figureNone.png
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,PMBOOKimgs/figureNone.png
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,PMBOOKimgs/figureNone.png
...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,PMBOOKimgs/figureNone.png
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,PMBOOKimgs/figureNone.png
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,PMBOOKimgs/figureNone.png
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",PMBOOKimgs/figureNone.png


In [12]:
# Define a function to replace a specific value with None
def replace_value_with_none(value, to_replace):
    if value == to_replace:
        return None
    else:
        return value

# Specify the value to be replaced with None
value_to_replace = "PMBOOKimgs/figureNone.png"

# Apply the function to the "Figure_URL" column
df['Figure'] = df['Figure'].apply(lambda x: replace_value_with_none(x, value_to_replace))

# Display the DataFrame with replaced values
df

,Number,Concept,Description,Described In,Figure
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None
...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None


In [13]:
def clean_text(text):
    # Remove special characters and symbols (retain letters, numbers, and basic punctuation)
    cleaned_text = re.sub(r'[^a-zA-Z0-9.,!? ]', '', text)
    
    # Remove extra spaces and trim leading/trailing spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

In [14]:
df['Definition'] = df['Description'].apply(clean_text)

In [15]:
df

,Number,Concept,Description,Described In,Figure,Definition
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,the process of defining how to conduct risk ma...
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,the process of identifying individual project ...
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,the process of prioritizing individual project...
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,the process ofnumerically analyzing the combin...
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,"the process of developing options, selecting s..."
...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,. work performance information includes inform...
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,y. the monitor risks process may result in a c...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,change to the project management plan goes thr...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,project documents that may be updated as a res...


In [16]:
from keybert import KeyBERT
import nltk
from nltk.tree import Tree
from nltk.corpus import wordnet
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
from nltk.corpus import wordnet

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))
    return list(synonyms)

def get_multiword_synonyms(phrase, top_n=6):
    words = phrase.split()
    synonyms_list = [get_synonyms(word) for word in words]
    
    # Generate all possible combinations of synonyms
    synonym_combinations = [', '.join(combination) for combination in itertools.product(*synonyms_list)]
    
    # Remove duplicates
    unique_synonyms = list(set(synonym_combinations))
    
        # Replace ", " with " " in each synonym
    cleaned_synonyms = [synonym.replace(", ", " ") for synonym in unique_synonyms]
    
    return cleaned_synonyms[:top_n]

In [37]:
df['Synonyms'] = df['Concept'].apply(get_multiword_synonyms)

In [38]:
df

,Number,Concept,Description,Described In,Figure,Definition,Synonyms
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,the process of defining how to conduct risk ma...,"[be after adventure direction, programme take ..."
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,the process of identifying individual project ...,"[key out put on the line, describe lay on the ..."
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,the process of prioritizing individual project...,"[do qualitative take chances analysis, do qual..."
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,the process ofnumerically analyzing the combin...,[do quantitative put on the line analytic thin...
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,"the process of developing options, selecting s...","[plan risk reception, project put on the line ..."
...,...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,. work performance information includes inform...,"[work out execution entropy, crop carrying int..."
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,y. the monitor risks process may result in a c...,"[variety petition, convert asking, deepen call..."
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,change to the project management plan goes thr...,"[task management plan update, protrude managem..."
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,project documents that may be updated as a res...,"[externalize written document update, throw wr..."


In [39]:
# Divide definition into sentences
df['sentences'] = df['Definition'].apply(nltk.sent_tokenize)

In [42]:
df.loc[6]["sentences"]

['the process of monitoring the implementation of agreedupon risk response planstracking identified risks, identifying and analyzing new risks, and evaluating risk process effectiveness throughoutthe projectfigure 111 provides an overview of the project risk management processes.',
 'the project management risk processesare presented as discrete processes with defined interfaces while, in practice, they overlap and interact in ways thatcannot be completely detailed in this pmboke guide.395ctionepifonotkey concepts for project risk managementail projects are risky since they are unique undertakings with varying degrees of complexity that aim to deliverbenefits they do this in a context of constraints and assumptions, while responding to stakeholder expectations thatmay be conflicting and changing organizations should choose to take project risk in a controlled and intentional mannerin order to create value while balancing risk and reward.project risk management aims to identify and mana

In [43]:
# Function to extract keywords from a sentence
def extract_keywords(sentence):
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    keywords = kw_model.extract_keywords(sentence, keyphrase_ngram_range=(2, 3), stop_words='english', highlight=False, top_n=2)
    return list(dict(keywords).keys())

# Function to calculate cosine similarity
def cosine_similarity_score(text1, text2):
    vectorizer = CountVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    return cosine_similarity([vectors[0]], [vectors[1]])[0][0]

In [ ]:
 # Process each sentence
words=df.loc[6]["Synonyms"]
words=words.append(df.loc[6]["Title"])
for sentence in df.loc[6]["sentences"]:
    # Extract keywords
    keywords = extract_keywords(sentence)
            
    test=False
    for word in keywords:
    for keyword in words:
        print("bbb")
        if cosine_similarity_score(word, keyword)>0.3:
            print("aaaaaaa")
            test=True
            

    # Check if similarity score is above a threshold
    if test:
        # Add to column keywords
        df.at[index, "Keywords"] = ", ".join(keywords)
    else:
        # Add a new line with extracted keywords as title and the sentence into description
        new_row = {"Title": ", ".join(keywords), "Definition": sentence}
        df = df.append(new_row, ignore_index=True)


In [49]:
words =["risk manegemnt","planning"]
list= ["operations", "extraction","simplified risk"]
test=False
for word in words:
    for keyword in list:
        print("bbb")
        if cosine_similarity_score(word, keyword)>0.3:
            test=True

bbb
bbb
bbb
aaaaaaaaaaaa
bbb
bbb
bbb
